In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import  Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.common.by import  By 
from selenium.webdriver.support import  expected_conditions as EC 

import pandas as pd
import numpy as np
import time
import datetime

In [20]:
#建立数据库
columns=['时间','发布日期','作者名','作者类型','粉丝数','投稿数','视频名','分区','子分区','视频类型','自制or转载','标签','视频长度','播放量','点赞数','投币数','收藏数','转发数','评论数','弹幕数','有无分集']
channel_donghua_MAD·AMV=pd.DataFrame(columns=columns)
channel_donghua_MAD·AMV

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集


In [25]:
driver=webdriver.Chrome()
driver.get('https://passport.bilibili.com/login') 

In [26]:
#登录
driver.find_element_by_xpath('//*[@id="login-username"]').send_keys('15717506191')
driver.find_element_by_xpath('//*[@id="login-passwd"]').send_keys('xy000000')
driver.find_element_by_xpath('//*[@id="geetest-wrap"]/div/div[5]/a[1]').click()
#需要手动过验证码

In [27]:
driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[1]/div/a/span').click()  #选择“动画”频道

In [28]:
#获得分区名
channel=driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[1]/div/a/span').text
channel=channel.strip('0123456789+')
channel

'动画'

In [29]:
driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[2]/a').click()  #选择“MAD·AMV”子频道
sub_channel=driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[2]/a').text
sub_channel

'MAD·AMV'

In [30]:
driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[1]/ul/a[2]/li').click() #让视频按热度排序

In [31]:
print(datetime.datetime.now())

for month in range(8,11): 
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div/span').click()
    period=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).text
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).click()
    WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]/div/div[2]/div[3]/a'))) 
    for page in range(5):    #爬取每个频道1月到10月每个月播放量前100的视频

        video_loc=1
        while video_loc<21:

            ### 判断作者类型###
            
            
            driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).click()
            driver.switch_to_window(driver.window_handles[-1])
            
            
            author_web_show='No'
            try:
                i=0
                while author_web_show=='No':
                    driver.refresh()
                    i+=1
                    try:
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="n-fs"]')))  #看作者名是否出现
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="navigator"]/div/div[1]/div[1]/a[3]/span[3]')))  #看投稿数是否出现
                        author_web_show='Yes'
                    except:
                        author_web_show='No' 
                        if i>3:  
                            raise Exception  #刷新几次后仍爬不了说明用户已注销

                try:
                    driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m organization-auth"]')
                    author_type='机构认证'

                except:
                    try:
                        driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m personal-auth"]')
                        author_type='个人认证'
                    except:
                        author_type='无'

                fans_num=driver.find_element_by_xpath('//*[@id="n-fs"]').text  #获取作者粉丝数

                ### 获取作者投稿数 ###
                driver.find_element_by_xpath('//*[@class="n-num"]').click()
                WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span')))  
                video_num=driver.find_element_by_xpath('//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span').text

                driver.close()
                driver.switch_to_window(driver.window_handles[-1])


                view_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[1]/span'%video_loc).text #获取播放量
                real_dan_mu_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[2]/span'%video_loc).text #获取弹幕数
                author=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).text #获取作者名
                driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/a'%video_loc).click() #进入视频界面  
                driver.switch_to_window(driver.window_handles[-1]) 

                try:
                    WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))  #若出现视频被删除的情况则退出

                    video_web_show='No'
                    while video_web_show=='No':
                        driver.refresh()

                        try:
                            WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@class="bilibili-player-video-time-total"]'))) #以视频长度出现时间判定网页是否加载完
                            video_web_show='Yes'

                            try:      
                                driver.find_element_by_xpath('//*[@id="multi_page"]')   #测试是否有分集视频
                                multi='是'

                            except:
                                multi='否'

                            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
                            driver.execute_script('window.scrollTo(document.body.scrollHeight,0)') #再调回最上面是因为，视频长度的元素必须在屏幕内才可以被提取，否则为空

                            bv=driver.current_url[33:44] # 获取BV号（BV号在网址中）
                            print('开始BV号为%s视频属性的抓取'%bv)
                            video_name=driver.find_element_by_xpath('//*[@id="viewbox_report"]/h1/span').text #获取视频名
                            
                            ### 获取发布日期 ###
                            release_date=driver.find_element_by_xpath('//*[@id="viewbox_report"]/div/span[3]').text
                            
                            ### 获取视频类型 ###
                            try:
                                video_type=driver.find_element_by_xpath('//*[@class="activity"]').text
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type=video_type+'和合作视频'
                                except:
                                    pass
                            except:
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type='合作视频'
                                except:
                                    video_type='普通视频'


                            ### 获取点赞数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text=='点赞':
                                like_num=0
                            else:
                                like_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text 

                            ### 获取投币数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text=='投币':
                                coin_num=0
                            else:
                                coin_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text

                            ### 获取收藏数 ###             
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text=='收藏':
                                collect_num=0
                            else:
                                collect_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text

                            ### 获取转发数 ###       
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text=='分享':
                                forward_num=0
                            else:
                                forward_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text

                            ### 获取视频长度 ###

                            video_length=driver.find_element_by_xpath('//*[@class="bilibili-player-video-time-total"]').text  

                            ### 判断自制还是转载 ###
                            driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').click()
                            try:
                                driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/div[2]/div[2]').text
                                self_made='自制'
                            except:
                                self_made='转载'
                            driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/i').click()

                            ### 获取标签 ###
                            k=1
                            labels=np.array
                            while True:
                                try:
                                    labels=np.append(labels,driver.find_element_by_xpath('//*[@id="v_tag"]/ul/li[%d]'%k).text)
                                    k+=1
                                except: 
                                    break
                            labels=labels[1:]

                            ### 获取评论数 ###
                            driver.execute_script('window.scrollTo(0, 700)')        
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[2]'))) #保证"评论"二字能加载出来                                                   
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div')))
                            if driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/div/div[4]/div').text=='没有更多信息':    #应对评论区被关闭的情况
                                comment_num='NaN'
                            else:
                                while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                    try:
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div[1]')))
                                        while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                            time.sleep(0.1) #很奇怪，评论数可以定位的一瞬间并不能爬取，会返回空
                                        comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text     
                                    except:
                                        driver.refresh()    #有的时候“评论”二字出现，评论数不出现，页面好像卡住了
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))
                                        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[1]')))

                                comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text

                            channel_donghua_MAD·AMV.loc[bv]=[period,release_date,author,author_type,fans_num,video_num,video_name,channel,sub_channel,video_type,self_made,labels,video_length,view_num,like_num,coin_num,collect_num,forward_num,comment_num,real_dan_mu_num,multi]

                            print('BV号为%s的视频属性抓取结束'%bv)

                            driver.close()
                            driver.switch_to_window(driver.window_handles[-1])

                        except:
                            video_web_show='No' 
                except:
                    driver.close()
                    driver.switch_to_window(driver.window_handles[-1])
            except:
                driver.close()
                driver.switch_to_window(driver.window_handles[-1])
            
            video_loc+=1


        driver.find_element_by_xpath('//*[@class="nav-btn iconfont icon-arrowdown3"]').click() #点击下一页
        WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]')))  #点击下一页后等页面刷新
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[2]').click()  #回到第1页
    WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]')))
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]').click()  #发现很多月份漏爬了约20个，而且恰好是第一页，所以再点一次第一页
        
        
print(datetime.datetime.now()) 
    

2020-12-19 09:37:35.269286


<ipython-input-31-864a38754432>:17: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-31-864a38754432>:54: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-31-864a38754432>:61: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为11A411Y7sX视频属性的抓取
BV号为11A411Y7sX的视频属性抓取结束


<ipython-input-31-864a38754432>:181: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为15h411o7KX视频属性的抓取
BV号为15h411o7KX的视频属性抓取结束
开始BV号为1Va4y1E7MS视频属性的抓取
BV号为1Va4y1E7MS的视频属性抓取结束
开始BV号为1Kp4y1v75k视频属性的抓取
BV号为1Kp4y1v75k的视频属性抓取结束
开始BV号为1LA411n7po视频属性的抓取
BV号为1LA411n7po的视频属性抓取结束
开始BV号为1PA411e75q视频属性的抓取
BV号为1PA411e75q的视频属性抓取结束
开始BV号为1Gt4y1S7Jv视频属性的抓取
BV号为1Gt4y1S7Jv的视频属性抓取结束
开始BV号为1R64y1c787视频属性的抓取
BV号为1R64y1c787的视频属性抓取结束
开始BV号为1wv411i7EH视频属性的抓取
BV号为1wv411i7EH的视频属性抓取结束
开始BV号为1FK411K7Di视频属性的抓取
BV号为1FK411K7Di的视频属性抓取结束
开始BV号为1e54y1i7b3视频属性的抓取
BV号为1e54y1i7b3的视频属性抓取结束
开始BV号为1354y1v7sp视频属性的抓取
BV号为1354y1v7sp的视频属性抓取结束
开始BV号为1Hv411i7xZ视频属性的抓取
BV号为1Hv411i7xZ的视频属性抓取结束
开始BV号为1pi4y1M7if视频属性的抓取
BV号为1pi4y1M7if的视频属性抓取结束
开始BV号为12k4y127GX视频属性的抓取
BV号为12k4y127GX的视频属性抓取结束
开始BV号为1Mt4y1S7qe视频属性的抓取
BV号为1Mt4y1S7qe的视频属性抓取结束
开始BV号为1et4y1D7Cf视频属性的抓取
BV号为1et4y1D7Cf的视频属性抓取结束
开始BV号为1hV411U72A视频属性的抓取
BV号为1hV411U72A的视频属性抓取结束
开始BV号为1b64y1c7oX视频属性的抓取
BV号为1b64y1c7oX的视频属性抓取结束
开始BV号为15f4y197cD视频属性的抓取
BV号为15f4y197cD的视频属性抓取结束
开始BV号为1Qv411y7jk视频属性的抓取
BV号为1Qv411y7jk的视频属性抓取结束
开始BV号为1RK4y1v7Tj视频属性的抓取
BV号为1RK4y1v7Tj的视

BV号为1Qy4y1y7zH的视频属性抓取结束
开始BV号为1MK4y1a7ss视频属性的抓取
BV号为1MK4y1a7ss的视频属性抓取结束
开始BV号为1LT4y1A75G视频属性的抓取
BV号为1LT4y1A75G的视频属性抓取结束
开始BV号为1Xy4y1y7QX视频属性的抓取
BV号为1Xy4y1y7QX的视频属性抓取结束
开始BV号为1oz4y1o7FB视频属性的抓取
BV号为1oz4y1o7FB的视频属性抓取结束
开始BV号为1PZ4y1T7HS视频属性的抓取
BV号为1PZ4y1T7HS的视频属性抓取结束
开始BV号为1nV411m7J7视频属性的抓取
BV号为1nV411m7J7的视频属性抓取结束
开始BV号为1SA411E7L1视频属性的抓取
BV号为1SA411E7L1的视频属性抓取结束
开始BV号为1DV41117dn视频属性的抓取
BV号为1DV41117dn的视频属性抓取结束
开始BV号为1Ki4y1u75g视频属性的抓取
BV号为1Ki4y1u75g的视频属性抓取结束
开始BV号为1ta4y1j7ap视频属性的抓取
BV号为1ta4y1j7ap的视频属性抓取结束
开始BV号为1M54y1v7pu视频属性的抓取
BV号为1M54y1v7pu的视频属性抓取结束
开始BV号为1Ep4y1a76H视频属性的抓取
BV号为1Ep4y1a76H的视频属性抓取结束
开始BV号为1yD4y1o7Yq视频属性的抓取
BV号为1yD4y1o7Yq的视频属性抓取结束
开始BV号为1RD4y1d7F4视频属性的抓取
BV号为1RD4y1d7F4的视频属性抓取结束
开始BV号为1nK411K78N视频属性的抓取
BV号为1nK411K78N的视频属性抓取结束
开始BV号为1Zz4y1o7uD视频属性的抓取
BV号为1Zz4y1o7uD的视频属性抓取结束
开始BV号为1KD4y1o7kT视频属性的抓取
BV号为1KD4y1o7kT的视频属性抓取结束
开始BV号为1Yk4y1y7sk视频属性的抓取
BV号为1Yk4y1y7sk的视频属性抓取结束
开始BV号为1wz4y1Z7YV视频属性的抓取
BV号为1wz4y1Z7YV的视频属性抓取结束
开始BV号为1eK4y1a7eW视频属性的抓取
BV号为1eK4y1a7eW的视频属性抓取结束
开始BV号为1Hy4y1y71Z

In [21]:
driver.switch_to_window(driver.window_handles[-1])

<ipython-input-21-dd0a1399d54d>:1: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


In [32]:
channel_donghua_MAD·AMV.groupby('时间').agg('count')

,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,自制or转载,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
时间,,,,,,,,,,,,,,,,,,,,
10月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
1月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
2月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
3月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
4月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
5月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
6月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
7月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
8月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100


In [33]:
channel_donghua_MAD·AMV.to_json('channel_donghua_MAD·AMV.json')